## Dependencies 

In [4]:
!pip install face_recognition 

     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 1.0/100.1 MB 8.5 MB/s eta 0:00:12
     - -------------------------------------- 2.6/100.1 MB 8.4 MB/s eta 0:00:12
     - -------------------------------------- 3.1/100.1 MB 8.0 MB/s eta 0:00:13
     - -------------------------------------- 3.4/100.1 MB 4.4 MB/s eta 0:00:23
     - -------------------------------------- 4.2/100.1 MB 4.9 MB/s eta 0:00:20
     -- ------------------------------------- 5.8/100.1 MB 4.9 MB/s eta 0:00:20
     -- ------------------------------------- 6.3/100.1 MB 5.1 MB/s eta 0:00:19
     -- ------------------------------------- 7.3/100.1 MB 4.9 MB/s eta 0:00:20
     --- ------------------------------------ 9.4/100.1 MB 5.2 MB/s eta 0:00:18
     ---- ---------------------------------- 11.5/100.1 MB 5.7 MB/s eta 0:00:16
     ----- --------------------------------- 12.8/100.1 MB 5.8 MB/s eta 0:00:15
     ----- --------------------------------- 14

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [41 lines of output]
  running bdist_wheel
  running build
  running build_ext
  
  
                     CMake is not installed on your system!
  
      Or it is possible some broken copy of cmake is installed on your system.
      It is unfortunately very common for python package managers to include
      broken copies of cmake.  So if the error above this refers to some file
      path to a cmake file inside a python or anaconda or miniconda path then you
      should delete that broken copy of cmake from your computer.
  
      Instead, please get an official copy of cmake from one of these known good
      sources of an official cmake:
          - cmake.org (this is how windows users should get cmake)
          - apt install cmake (for Ubuntu or Debian based systems)
          - yum install cmake (for Redhat or CenOS based systems)
  
      On a linux machine you can run `which cmake` to see what cmake you are
      act

In [6]:
pip install mediapipe

   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   - -------------------------------------- 1.6/51.0 MB 8.4 MB/s eta 0:00:06
   -- ------------------------------------- 3.7/51.0 MB 9.1 MB/s eta 0:00:06
   ---- ----------------------------------- 5.5/51.0 MB 9.3 MB/s eta 0:00:05
   ----- ---------------------------------- 7.3/51.0 MB 9.3 MB/s eta 0:00:05
   ------- -------------------------------- 9.4/51.0 MB 9.5 MB/s eta 0:00:05
   -------- ------------------------------- 11.3/51.0 MB 9.4 MB/s eta 0:00:05
   --------- ------------------------------ 12.6/51.0 MB 8.9 MB/s eta 0:00:05
   ----------- ---------------------------- 14.2/51.0 MB 8.8 MB/s eta 0:00:05
   ------------ --------------------------- 16.5/51.0 MB 9.1 MB/s eta 0:00:04
   -------------- ------------------------- 18.6/51.0 MB 9.2 MB/s eta 0:00:04
   ---------------- ----------------------- 20.7/51.0 MB 9.3 MB/s eta 0:00:04
   ------------------ --------------------- 23.1/51.0 MB 9.5 MB/s eta 0:00:03

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Esco\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25857 sha256=a8bee7d336691da755653fa5e4e42265e6a1bebed49a71b7c5820aec388ae58e
  Stored in directory: c:\users\esco\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install opencv-python

In [3]:
!pip install pandas

In [5]:
!pip install seaborn

## Original Code

In [ ]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import face_recognition
import numpy as np
import argparse
import imutils
import time
import cv2
from threading import Thread

def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # return the eye aspect ratio
    return ear

# manually set the webcam index
args = {"webcam": 0}  

# define two constants, one for the eye aspect ratio to indicate
# blink and another constant for the number of consecutive
# frames the eye must be below the threshold for to set off the alarm
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 15

# initialize the frame counter as well as a boolean used to
# indicate if the alarm is on
COUNTER = 0
ALARM_ON = False

# start the video stream thread
print("[INFO] starting video stream thread...")
vs = VideoStream(src=args["webcam"]).start()
time.sleep(1.0)

# loop over frames from the video stream
while True:
    # grab the frame from the threaded video file stream, resize it, and convert it to grayscale
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # detect face landmarks
    face_landmarks_list = face_recognition.face_landmarks(rgb_frame)

    # loop over the face detections
    for face_landmarks in face_landmarks_list:
        # extract the left and right eye coordinates
        leftEye = face_landmarks['left_eye']
        rightEye = face_landmarks['right_eye']

        # convert eye coordinates to NumPy arrays
        leftEye = np.array(leftEye)
        rightEye = np.array(rightEye)

        # compute the eye aspect ratio for both eyes
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0

        # compute the convex hull for the left and right eye, then visualize each of the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        # check to see if the eye aspect ratio is below the blink threshold
        if ear < EYE_AR_THRESH:
            COUNTER += 1

            # if the eyes were closed for a sufficient number of frames, trigger the alarm
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                # if the alarm is not on, turn it on
                if not ALARM_ON:
                    ALARM_ON = True

                # draw an alarm message on the frame
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # otherwise, the eye aspect ratio is not below the threshold, reset the counter and alarm
        else:
            COUNTER = 0
            ALARM_ON = False

        # draw the computed eye aspect ratio on the frame for debugging
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(5) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
vs.stop()
cv2.destroyAllWindows()

## Edited Version

In [3]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import face_recognition
import numpy as np
import argparse
import imutils
import time
import cv2
from threading import Thread

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    
    return ear

def mouth_aspect_ratio(mouth, face_bbox):
    A = dist.euclidean(mouth[2], mouth[10])  # Outer top lip (51) to outer bottom lip (59)
    face_height = face_bbox[2] - face_bbox[0]  # bottom - top
    mar = A / face_height  # Normalize MAR by face height

    return mar

# Manually set the webcam index
args = {"webcam": 0}

EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 10
MAR_THRESH = 0.4
MOUTH_AR_CONSEC_FRAMES = 5
FRAME_SKIP = 2  # Skip every 2nd frame
frame_count = 0

COUNTER = 0
ALARM_ON = False
face_landmarks_list = []

print("[INFO] starting video stream thread...")
vs = VideoStream(src=args["webcam"]).start()
time.sleep(1.0)

def detect_face_landmarks(rgb_frame):
    global face_landmarks_list
    face_landmarks_list = face_recognition.face_landmarks(rgb_frame)

while True:
    frame = vs.read()
    
    # Increment frame count for skipping frames logic
    frame_count += 1

    # Skip frames based on FRAME_SKIP setting
    if frame_count % FRAME_SKIP != 0:
        continue  # Skip this frame and move to the next

    # After skipping, continue with processing the current frame
    frame = imutils.resize(frame, width=720)  # Reduce resolution
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Start face detection in a separate thread for the current frame
    face_thread = Thread(target=detect_face_landmarks, args=(rgb_frame,))
    face_thread.start()
    
    for (top, right, bottom, left), face_landmarks in zip(face_locations, face_landmarks_list):
        # Extract the face bounding box
        face_bbox = (left, top, right, bottom)

        # Extract the landmarks
        leftEye = np.array(face_landmarks['left_eye'])
        rightEye = np.array(face_landmarks['right_eye'])
        mouth = np.array(face_landmarks['top_lip'] + face_landmarks['bottom_lip'])

        # Compute EAR (Eye Aspect Ratio
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        # Compute MAR (Mouth Aspect Ratio) with face bounding box
        mar = mouth_aspect_ratio(mouth, face_bbox)

        #leftEyeHull = cv2.convexHull(leftEye)
        #rightEyeHull = cv2.convexHull(rightEye)
        #cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        #cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        warning_text_1 = ""
        warning_text_2 = ""
        
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                ALARM_ON = True
                warning_text_1 = "DROWSINESS ALERT!"
        
        else:
            COUNTER = 0
            ALARM_ON = False
        
        if mar > MAR_THRESH:
            COUNTER += 1
            if COUNTER >= MOUTH_AR_CONSEC_FRAMES:
                ALARM_ON = True
            warning_text_2 = "YAWNING DETECTED!"
        
        if warning_text_1:
            cv2.putText(frame, warning_text_1, (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        if warning_text_2:
            cv2.putText(frame, warning_text_2, (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Re-add EAR and MAR display
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "MAR: {:.2f}".format(mar), (300, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(5) & 0xFF

    # if the q key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
vs.stream.release()
vs.stop()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'face_recognition'

## Mediapipe model

In [7]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import time

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Constants
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 15
MAR_THRESH = 0.09# Adjust based on testing
MAR_COUNTER = 0  # Add a separate counter for yawning
MAR_CONSEC_FRAMES = 15
FRAME_SKIP = 2  # Skip every 2nd frame to improve performance
frame_count = 0
COUNTER = 0
ALARM_ON = False


# Counters for warnings
drowsiness_count = 0
yawning_count = 0

# Store timestamps of warnings
drowsiness_timestamps = []
yawning_timestamps = []

# EAR Calculation
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# MAR Calculation (using outer lip landmarks only)
def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[0], mouth[1])  # Outer top lip (point 13) to bottom lip (point 14)
    return A  # No need to normalize with face height in this case

# Start video stream
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    frame = vs.read()
    frame_count += 1

    # Skip frames based on FRAME_SKIP setting
    if frame_count % FRAME_SKIP != 0:
        continue  # Skip this frame and move to the next

    frame = cv2.flip(frame, 1)  # Flip for a mirror effect
    frame = cv2.resize(frame, (720, 480))
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect facial landmarks
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract Eye and Mouth landmarks
            landmarks = face_landmarks.landmark

            # Eye indices based on MediaPipe Face Mesh
            left_eye_idx = [33, 160, 158, 133, 153, 144]  # Example points
            right_eye_idx = [263, 387, 385, 362, 380, 373]

            leftEye = np.array([(landmarks[i].x, landmarks[i].y) for i in left_eye_idx])
            rightEye = np.array([(landmarks[i].x, landmarks[i].y) for i in right_eye_idx])

            # Mouth indices (using outer lips)
            mouth_idx = [13, 14]  # Outer top and bottom lips
            mouth = np.array([(landmarks[i].x, landmarks[i].y) for i in mouth_idx])

            # Compute EAR & MAR
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            # Debugging: Print EAR and MAR
            #print(f"EAR: {ear:.2f}, MAR: {mar:.2f}")

            # Display Warnings
            warning_text_1 = ""
            warning_text_2 = ""

            if ear < EYE_AR_THRESH:
                COUNTER += 1
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    ALARM_ON = True
                    warning_text_1 = "DROWSINESS ALERT!"

                # Log the occurrence
                drowsiness_count += 1
                drowsiness_timestamps.append(time.strftime("%H:%M:%S"))
                
            else:
                COUNTER = 0
                ALARM_ON = False

            if mar > MAR_THRESH:
                MAR_COUNTER += 1
                if MAR_COUNTER >= MAR_CONSEC_FRAMES:
                    warning_text_2 = "YAWNING DETECTED!"
                    
                # Log the occurrence
                yawning_count += 1
                yawning_timestamps.append(time.strftime("%H:%M:%S"))
                
            else:
                MAR_COUNTER = 0

            if warning_text_1:
                cv2.putText(frame, warning_text_1, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            if warning_text_2:
                cv2.putText(frame, warning_text_2, (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Display EAR & MAR values
            cv2.putText(frame, f"EAR: {ear:.2f}", (500, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (500, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show frame
    cv2.imshow("Fatigue Detection", frame)
    key = cv2.waitKey(5) & 0xFF

    # Exit if 'q' is pressed
    if key == ord("q"):
        break

print("\n========== FATIGUE REPORT ==========")
print(f"Total Drowsiness Alerts: {drowsiness_count}")
print(f"Total Yawning Alerts: {yawning_count}")

if drowsiness_count > 0:
    print("\nDrowsiness detected at:")
    for timestamp in drowsiness_timestamps:
        print(f" - {timestamp}")

if yawning_count > 0:
    print("\nYawning detected at:")
    for timestamp in yawning_timestamps:
        print(f" - {timestamp}")

print("\n====================================")

# Cleanup
vs.stream.release()
vs.stop()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe'

## Mediapipe report fixed

In [1]:
import cv2
import pandas as pd
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist
from imutils.video import VideoStream

import time

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Constants
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 15
MAR_THRESH = 0.09
MAR_CONSEC_FRAMES = 15
FRAME_SKIP = 2

# Counters and flags
frame_count = 0
COUNTER = 0
MAR_COUNTER = 0
ALARM_ON = False
yawn_alarm_on = False

# Log counters and timestamps
drowsiness_count = 0
yawning_count = 0
drowsiness_timestamps = []
yawning_timestamps = []

# EAR Calculation
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# MAR Calculation
def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[0], mouth[1])
    return A

# Start video stream
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    frame = vs.read()
    frame_count += 1

    if frame_count % FRAME_SKIP != 0:
        continue

    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (720, 480))
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            left_eye_idx = [33, 160, 158, 133, 153, 144]
            right_eye_idx = [263, 387, 385, 362, 380, 373]
            mouth_idx = [13, 14]

            leftEye = np.array([(landmarks[i].x, landmarks[i].y) for i in left_eye_idx])
            rightEye = np.array([(landmarks[i].x, landmarks[i].y) for i in right_eye_idx])
            mouth = np.array([(landmarks[i].x, landmarks[i].y) for i in mouth_idx])

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            warning_text_1 = ""
            warning_text_2 = ""

            if ear < EYE_AR_THRESH:
                COUNTER += 1
                if COUNTER == EYE_AR_CONSEC_FRAMES:
                    ALARM_ON = True
                    drowsiness_count += 1
                    drowsiness_timestamps.append(time.strftime("%H:%M:%S"))
                    warning_text_1 = "DROWSINESS ALERT!"
                elif COUNTER > EYE_AR_CONSEC_FRAMES:
                    warning_text_1 = "DROWSINESS ALERT!"
            else:
                COUNTER = 0
                ALARM_ON = False

            if mar > MAR_THRESH:
                MAR_COUNTER += 1
                if MAR_COUNTER == MAR_CONSEC_FRAMES:
                    yawn_alarm_on = True
                    yawning_count += 1
                    yawning_timestamps.append(time.strftime("%H:%M:%S"))
                    warning_text_2 = "YAWNING DETECTED!"
                elif MAR_COUNTER > MAR_CONSEC_FRAMES:
                    warning_text_2 = "YAWNING DETECTED!"
            else:
                MAR_COUNTER = 0
                yawn_alarm_on = False

            if warning_text_1:
                cv2.putText(frame, warning_text_1, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            if warning_text_2:
                cv2.putText(frame, warning_text_2, (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            cv2.putText(frame, f"EAR: {ear:.2f}", (500, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (500, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Fatigue Detection", frame)
    key = cv2.waitKey(5) & 0xFF
    if key == ord("q"):
        break

# Report
print("\n========== FATIGUE REPORT ==========")
print(f"Total Drowsiness Alerts: {drowsiness_count}")
print(f"Total Yawning Alerts: {yawning_count}")

if drowsiness_timestamps:
    print("\nDrowsiness detected at:")
    for t in drowsiness_timestamps:
        print(f" - {t}")

if yawning_timestamps:
    print("\nYawning detected at:")
    for t in yawning_timestamps:
        print(f" - {t}")
print("====================================")


data = {
    "Time": drowsiness_timestamps + yawning_timestamps,
    "Warning": ["Drowsiness Alert"] * len(drowsiness_timestamps) + ["Yawning Alert"] * len(yawning_timestamps)
}

df = pd.DataFrame(data)
df.to_csv("fatigue_report.csv", index=False)

print("Report saved as fatigue_report.csv")

# Cleanup
vs.stream.release()
vs.stop()
cv2.destroyAllWindows()



========== FATIGUE REPORT ==========
Total Drowsiness Alerts: 0
Total Yawning Alerts: 0
Report saved as fatigue_report.csv


## Mediapipe report saved into excel and visual

In [9]:
import cv2
import pandas as pd
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Constants
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 15
MAR_THRESH = 0.09
MAR_CONSEC_FRAMES = 15
FRAME_SKIP = 2

# Counters and flags
frame_count = 0
COUNTER = 0
MAR_COUNTER = 0
ALARM_ON = False
yawn_alarm_on = False

# Log counters and timestamps
drowsiness_count = 0
yawning_count = 0
drowsiness_timestamps = []
yawning_timestamps = []

# EAR Calculation
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# MAR Calculation
def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[0], mouth[1])
    return A

# Start video stream
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    frame = vs.read()
    frame_count += 1

    if frame_count % FRAME_SKIP != 0:
        continue

    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (720, 480))
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            left_eye_idx = [33, 160, 158, 133, 153, 144]
            right_eye_idx = [263, 387, 385, 362, 380, 373]
            mouth_idx = [13, 14]

            leftEye = np.array([(landmarks[i].x, landmarks[i].y) for i in left_eye_idx])
            rightEye = np.array([(landmarks[i].x, landmarks[i].y) for i in right_eye_idx])
            mouth = np.array([(landmarks[i].x, landmarks[i].y) for i in mouth_idx])

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            warning_text_1 = ""
            warning_text_2 = ""

            if ear < EYE_AR_THRESH:
                COUNTER += 1
                if COUNTER == EYE_AR_CONSEC_FRAMES:
                    ALARM_ON = True
                    drowsiness_count += 1
                    drowsiness_timestamps.append(time.strftime("%Y-%m-%d %H:%M:%S"))
                    warning_text_1 = "DROWSINESS ALERT!"
                elif COUNTER > EYE_AR_CONSEC_FRAMES:
                    warning_text_1 = "DROWSINESS ALERT!"
            else:
                COUNTER = 0
                ALARM_ON = False

            if mar > MAR_THRESH:
                MAR_COUNTER += 1
                if MAR_COUNTER == MAR_CONSEC_FRAMES:
                    yawn_alarm_on = True
                    yawning_count += 1
                    yawning_timestamps.append(time.strftime("%Y-%m-%d %H:%M:%S"))
                    warning_text_2 = "YAWNING DETECTED!"
                elif MAR_COUNTER > MAR_CONSEC_FRAMES:
                    warning_text_2 = "YAWNING DETECTED!"
            else:
                MAR_COUNTER = 0
                yawn_alarm_on = False

            if warning_text_1:
                cv2.putText(frame, warning_text_1, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            if warning_text_2:
                cv2.putText(frame, warning_text_2, (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            cv2.putText(frame, f"EAR: {ear:.2f}", (500, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (500, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Fatigue Detection", frame)
    key = cv2.waitKey(5) & 0xFF
    if key == ord("q"):
        break

# Report
print("\n========== FATIGUE REPORT ==========")
print(f"Total Drowsiness Alerts: {drowsiness_count}")
print(f"Total Yawning Alerts: {yawning_count}")

if drowsiness_timestamps:
    print("\nDrowsiness detected at:")
    for t in drowsiness_timestamps:
        print(f" - {t}")

if yawning_timestamps:
    print("\nYawning detected at:")
    for t in yawning_timestamps:
        print(f" - {t}")
print("====================================")

# Save log to Excel and add a visual dashboard
data = {
    "timestamp": drowsiness_timestamps + yawning_timestamps,
    "event": ["Drowsiness Alert"] * len(drowsiness_timestamps) + ["Yawning Alert"] * len(yawning_timestamps)
}

df = pd.DataFrame(data)
df.to_excel("fatigue_log.xlsx", index=False)

# Create and save bar chart
# Load the data
df = pd.read_excel('fatigue_log.xlsx')
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%d %H:%M:%S")

# Count occurrences
counts = df['event'].value_counts()

# Create and save plot
plt.figure(figsize=(6, 4))
sns.barplot(x=counts.index, y=counts.values, hue=counts.index, palette="pastel", legend=False)
plt.title('Fatigue Event Counts')
plt.ylabel('Count')
plt.xlabel('Event Type')
plt.tight_layout()
plot_path = 'fatigue_plot.png'
plt.savefig(plot_path)  # Save the plot as an image file
plt.close()

# Create Excel file with image embedded
with pd.ExcelWriter('fatigue_dashboard.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Data', index=False)

    workbook = writer.book
    worksheet = workbook.add_worksheet('Dashboard')
    worksheet.insert_image('B2', plot_path)  # Insert plot at cell B2

print("Report saved and dashboard updated in fatigue_log.xlsx")

# Cleanup
vs.stream.release()
vs.stop()
cv2.destroyAllWindows()



========== FATIGUE REPORT ==========
Total Drowsiness Alerts: 1
Total Yawning Alerts: 0

Drowsiness detected at:
 - 2025-05-19 11:58:35
Report saved and dashboard updated in fatigue_log.xlsx


## IDing function added

In [1]:
import cv2
import pandas as pd
import mediapipe as mp
import numpy as np
from deepface import DeepFace
import os
import time
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Constants
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 15
MAR_THRESH = 0.09
MAR_CONSEC_FRAMES = 15
FRAME_SKIP = 2
DEEPFACE_CHECK_INTERVAL = 300  # every 300 frames (~10 seconds at 30fps)

# Counters and flags
frame_count = 0
COUNTER = 0
MAR_COUNTER = 0
ALARM_ON = False
yawn_alarm_on = False

# Log counters and timestamps
drowsiness_log = []
yawning_log = []
current_name = "Unknown"

# EAR Calculation
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# MAR Calculation
def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[0], mouth[1])
    return A

# Start video stream
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    frame = vs.read()
    frame_count += 1
    if frame_count % FRAME_SKIP != 0:
        continue

    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (720, 480))
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Face recognition using DeepFace every 300 frames
    if frame_count % DEEPFACE_CHECK_INTERVAL == 0:
        try:
            bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            result = DeepFace.find(
                img_path=bgr_frame,
                db_path="known_faces",
                model_name="Facenet",
                distance_metric="cosine",
                enforce_detection=False,
                silent=True,
                threshold=0.6
            )

            if len(result) > 0 and len(result[0]) > 0:
                identity = os.path.basename(result[0].iloc[0]['identity'])
                current_name = os.path.splitext(identity)[0]
                # Match found (silent)
            else:
                current_name = "Unknown"
                # No match found (silent)

        except Exception:
            current_name = "Unknown"
            # Face recognition failed (silent)

    # MediaPipe facial landmarks
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            left_eye_idx = [33, 160, 158, 133, 153, 144]
            right_eye_idx = [263, 387, 385, 362, 380, 373]
            mouth_idx = [13, 14]

            leftEye = np.array([(landmarks[i].x, landmarks[i].y) for i in left_eye_idx])
            rightEye = np.array([(landmarks[i].x, landmarks[i].y) for i in right_eye_idx])
            mouth = np.array([(landmarks[i].x, landmarks[i].y) for i in mouth_idx])

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            warning_text_1 = ""
            warning_text_2 = ""

            if ear < EYE_AR_THRESH:
                COUNTER += 1
                if COUNTER == EYE_AR_CONSEC_FRAMES:
                    ALARM_ON = True
                    drowsiness_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), current_name))
                    warning_text_1 = "DROWSINESS ALERT!"
                elif COUNTER > EYE_AR_CONSEC_FRAMES:
                    warning_text_1 = "DROWSINESS ALERT!"
            else:
                COUNTER = 0
                ALARM_ON = False

            if mar > MAR_THRESH:
                MAR_COUNTER += 1
                if MAR_COUNTER == MAR_CONSEC_FRAMES:
                    yawn_alarm_on = True
                    yawning_log.append((time.strftime("%Y-%m-%d %H:%M:%S"), current_name))
                    warning_text_2 = "YAWNING DETECTED!"
                elif MAR_COUNTER > MAR_CONSEC_FRAMES:
                    warning_text_2 = "YAWNING DETECTED!"
            else:
                MAR_COUNTER = 0
                yawn_alarm_on = False

            if warning_text_1:
                cv2.putText(frame, warning_text_1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if warning_text_2:
                cv2.putText(frame, warning_text_2, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            cv2.putText(frame, f"EAR: {ear:.2f}", (600, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (600, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show current recognized name with color based on match status
    name_color = (0, 255, 0) if current_name != "Unknown" else (0, 0, 255)
    cv2.putText(frame, current_name, (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, name_color, 2)

    cv2.imshow("Fatigue Detection", frame)
    if cv2.waitKey(5) & 0xFF == ord("q"):
        break

# Create and save report
data = {
    "timestamp": [ts for ts, _ in drowsiness_log + yawning_log],
    "person": [name for _, name in drowsiness_log + yawning_log],
    "event": ["Drowsiness Alert"] * len(drowsiness_log) + ["Yawning Alert"] * len(yawning_log)
}

df = pd.DataFrame(data)
df.to_excel("fatigue_log.xlsx", index=False)

# Create and save plot
counts = df['event'].value_counts()
plt.figure(figsize=(6, 4))
sns.barplot(x=counts.index, y=counts.values, hue=counts.index, palette="pastel", legend=False)
plt.title('Fatigue Event Counts')
plt.ylabel('Count')
plt.xlabel('Event Type')
plt.tight_layout()
plt.savefig("fatigue_plot.png")
plt.close()

# Plotting
# Convert timestamp column to datetime if not already
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Round timestamps to the nearest minute for grouping (or use .dt.floor('10s') for 10-sec bins)
df['minute'] = df['timestamp'].dt.floor('T')

# Group by time and event
time_counts = df.groupby(['minute', 'event']).size().reset_index(name='count')

# Create bar plot
plt.figure(figsize=(10, 5))
sns.barplot(data=time_counts, x='minute', y='count', hue='event', palette="Set2")
plt.xticks(rotation=45)
plt.title('Fatigue Events Over Time')
plt.ylabel('Event Count')
plt.xlabel('Time')
plt.tight_layout()
plt.savefig("fatigue_time_plot.png")
plt.close()

# Create unique filenames using timestamp
log_filename = f"fatigue_log_{timestamp}.xlsx"
dashboard_filename = f"fatigue_dashboard_{timestamp}.xlsx"

# Save the log file
df.to_excel(log_filename, index=False)

# Save Excel dashboard with embedded plot
with pd.ExcelWriter(dashboard_filename, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Data', index=False)
    workbook = writer.book
    worksheet = workbook.add_worksheet('Dashboard')
    worksheet.insert_image('B2', 'fatigue_plot.png')
    worksheet.insert_image('B22', 'fatigue_time_plot.png')  # adjust position if needed
    
print(f"Files saved as {log_filename} and {dashboard_filename}")

vs.stream.release()
vs.stop()
cv2.destroyAllWindows()

NameError: name 'timestamp' is not defined

## Recorded video timestamping 

In [96]:
import cv2
import pandas as pd
import mediapipe as mp
import numpy as np
from deepface import DeepFace
import os
import time
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Constants
EYE_AR_THRESH = 0.37
EYE_AR_CONSEC_FRAMES = 10
MAR_THRESH = 0.07
MAR_CONSEC_FRAMES = 10
FRAME_SKIP = 1
DEEPFACE_CHECK_INTERVAL = 300  # ~10 seconds at 30fps

# Counters and flags
frame_count = 0
COUNTER = 0
MAR_COUNTER = 0
ALARM_ON = False
yawn_alarm_on = False

# Log counters and timestamps
drowsiness_log = []
yawning_log = []
current_name = "Unknown"

# EAR Calculation
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

# MAR Calculation
def mouth_aspect_ratio(mouth):
    return dist.euclidean(mouth[0], mouth[1])

# Load video file
video_path = "C:/Users/yeou1/Fatigue_project/Accuracy/test_2.mp4"  # Update with your file path
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    exit()

# Get video FPS and compute frame delay
fps = cap.get(cv2.CAP_PROP_FPS)
frame_delay = int(1000 / fps) if fps > 0 else 33  # default to ~30fps

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % FRAME_SKIP != 0:
        continue

    frame = cv2.resize(frame, (720, 480))
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # DeepFace ID every N frames
    if frame_count % DEEPFACE_CHECK_INTERVAL == 0:
        try:
            bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            result = DeepFace.find(
                img_path=bgr_frame,
                db_path="known_faces",
                model_name="Facenet",
                distance_metric="cosine",
                enforce_detection=False,
                silent=True,
                threshold=0.6
            )
            if len(result) > 0 and len(result[0]) > 0:
                identity = os.path.basename(result[0].iloc[0]['identity'])
                current_name = os.path.splitext(identity)[0]
            else:
                current_name = "Unknown"
        except Exception:
            current_name = "Unknown"

    # MediaPipe facial landmarks
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            left_eye_idx = [33, 160, 158, 133, 153, 144]
            right_eye_idx = [263, 387, 385, 362, 380, 373]
            mouth_idx = [13, 14]

            leftEye = np.array([(landmarks[i].x, landmarks[i].y) for i in left_eye_idx])
            rightEye = np.array([(landmarks[i].x, landmarks[i].y) for i in right_eye_idx])
            mouth = np.array([(landmarks[i].x, landmarks[i].y) for i in mouth_idx])

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            mar = mouth_aspect_ratio(mouth)

            # Get video timestamp
            current_time_sec = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
            timestamp = time.strftime("%H:%M:%S", time.gmtime(current_time_sec))

            if ear < EYE_AR_THRESH:
                COUNTER += 1
                if COUNTER == EYE_AR_CONSEC_FRAMES:
                    ALARM_ON = True
                    drowsiness_log.append((timestamp, current_name))
            else:
                COUNTER = 0
                ALARM_ON = False

            if mar > MAR_THRESH:
                MAR_COUNTER += 1
                if MAR_COUNTER == MAR_CONSEC_FRAMES:
                    yawn_alarm_on = True
                    yawning_log.append((timestamp, current_name))
            else:
                MAR_COUNTER = 0
                yawn_alarm_on = False

            # Overlay data on frame (optional for debugging)
            cv2.putText(frame, f"EAR: {ear:.2f}", (600, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"MAR: {mar:.2f}", (600, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    name_color = (0, 255, 0) if current_name != "Unknown" else (0, 0, 255)
    cv2.putText(frame, current_name, (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, name_color, 2)

    # Show video during processing
    cv2.imshow("Fatigue Detection (Video)", frame)
    if cv2.waitKey(frame_delay) & 0xFF == ord("q"):
        break

# Save fatigue event logs
data = {
    "timestamp": [ts for ts, _ in drowsiness_log + yawning_log],
    "person": [name for _, name in drowsiness_log + yawning_log],
    "event": ["Drowsiness Alert"] * len(drowsiness_log) + ["Yawning Alert"] * len(yawning_log)
}
df = pd.DataFrame(data)
df.to_excel("fatigue_log.xlsx", index=False)

# Plot event counts
counts = df['event'].value_counts()
plt.figure(figsize=(6, 4))
sns.barplot(x=counts.index, y=counts.values, hue=counts.index, palette="pastel", legend=False)
plt.title('Fatigue Event Counts')
plt.ylabel('Count')
plt.xlabel('Event Type')
plt.tight_layout()
plt.savefig("fatigue_plot.png")
plt.close()

# Save Excel dashboard
with pd.ExcelWriter('fatigue_dashboard.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Data', index=False)
    workbook = writer.book
    worksheet = workbook.add_worksheet('Dashboard')
    worksheet.insert_image('B2', 'fatigue_plot.png')

print("Dashboard saved as fatigue_dashboard.xlsx")

cap.release()
cv2.destroyAllWindows()


Dashboard saved as fatigue_dashboard.xlsx


## Accuracy test

In [7]:
import pandas as pd
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix

# Load files
ground_df = pd.read_excel("ground_truth.xlsx") #the file is in Accuracy folder
pred_df = pd.read_excel("fatigue_log.xlsx") #load the excel file produced from the timestamping

def timestamp_to_sec(t):
    # If it's already a datetime.time object
    if hasattr(t, "hour"):
        return t.hour * 3600 + t.minute * 60 + t.second
    # If it's a string like "HH:MM:SS"
    try:
        parsed_time = datetime.strptime(t, "%H:%M:%S").time()
        return parsed_time.hour * 3600 + parsed_time.minute * 60 + parsed_time.second
    except:
        return 0  # or raise error/log if unexpected format

ground_df['seconds'] = ground_df['timestamp'].apply(timestamp_to_sec)
pred_df['seconds'] = pred_df['timestamp'].apply(timestamp_to_sec)
# Match flags
matched_preds = set()
y_true = []
y_pred = []

# Tolerance window
TOLERANCE = 2

# Match ground truth to predictions
for idx, row in ground_df.iterrows():
    gt_time = row['seconds']
    gt_name = row['person']
    gt_event = row['event']

    # Check for a matching prediction within ±2 sec
    match = pred_df[
        (abs(pred_df['seconds'] - gt_time) <= TOLERANCE) &
        (pred_df['person'] == gt_name) &
        (pred_df['event'] == gt_event)
    ]

    if not match.empty:
        matched_idx = match.index[0]
        matched_preds.add(matched_idx)
        y_true.append('fatigue')
        y_pred.append('fatigue')  # True Positive
    else:
        y_true.append('fatigue')
        y_pred.append('no_fatigue')  # False Negative

# Check for unmatched predictions (False Positives)
for idx, row in pred_df.iterrows():
    if idx not in matched_preds:
        y_true.append('no_fatigue')
        y_pred.append('fatigue')  # False Positive

# Compute and display metrics
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred, labels=['fatigue', 'no_fatigue']))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, labels=['fatigue', 'no_fatigue']))


Confusion Matrix:
[[ 8  2]
 [12  0]]

Classification Report:
              precision    recall  f1-score   support

     fatigue       0.40      0.80      0.53        10
  no_fatigue       0.00      0.00      0.00        12

    accuracy                           0.36        22
   macro avg       0.20      0.40      0.27        22
weighted avg       0.18      0.36      0.24        22

